In [1328]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [1329]:
df = pd.read_csv('diabetes.csv') 
print(df.shape)
df.describe().transpose()

(768, 9)


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [1330]:
target_column = ['Outcome'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,0.226180,0.198210,0.000000,0.058824,0.176471,0.352941,1.0
Glucose,768.0,0.607510,0.160666,0.000000,0.497487,0.587940,0.704774,1.0
BloodPressure,768.0,0.566438,0.158654,0.000000,0.508197,0.590164,0.655738,1.0
SkinThickness,768.0,0.207439,0.161134,0.000000,0.000000,0.232323,0.323232,1.0
Insulin,768.0,0.094326,0.136222,0.000000,0.000000,0.036052,0.150414,1.0
BMI,768.0,0.476790,0.117499,0.000000,0.406855,0.476900,0.545455,1.0
DiabetesPedigreeFunction,768.0,0.194990,0.136913,0.032231,0.100723,0.153926,0.258781,1.0
Age,768.0,0.410381,0.145188,0.259259,0.296296,0.358025,0.506173,1.0
Outcome,768.0,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.0


In [1331]:
X = df[predictors].values
y = df[target_column].values
test_size=0.30
develop_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=40)
X_trainnodev = X_train
X_testnodev = X_test
y_trainnodev = y_train
y_testnodev = y_test
newX_train = X_train
newX_test = X_test
newy_train = y_train
newy_test = y_test
post_split_develop_size = develop_size / (1 - test_size)
X_train, X_develop, y_train, y_develop = train_test_split(X_train, y_train, test_size=post_split_develop_size, random_state=40)
print(X_train.shape); print(X_test.shape); print(X_develop.shape)

(383, 8)
(231, 8)
(154, 8)


In [1332]:
(8 + 1)/2

4.5

In [1333]:
lrc = LogisticRegression(random_state=0, max_iter=1000 ,verbose = True)
lrc.fit(newX_train, newy_train.ravel())
regpred_train = lrc.predict(newX_train)
regpred_test = lrc.predict(newX_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [1334]:
print("MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA");print(confusion_matrix(newy_test,regpred_test))
print(classification_report(newy_test,regpred_test))

MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA
[[129  13]
 [ 46  43]]
              precision    recall  f1-score   support

           0       0.74      0.91      0.81       142
           1       0.77      0.48      0.59        89

    accuracy                           0.74       231
   macro avg       0.75      0.70      0.70       231
weighted avg       0.75      0.74      0.73       231



In [1335]:
mlp = MLPClassifier( max_iter=2000)
mlpdev = MLPClassifier( max_iter=4000)

In [1336]:
parameter_space = {
    'hidden_layer_sizes': [(5,5,5),(4,4,4), (8,8,8)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'learning_rate': ['constant','adaptive']
    }
random_search = RandomizedSearchCV(mlp, parameter_space,n_iter = 12, cv = 5)
random_search.fit(X_trainnodev,y_trainnodev.ravel())
params = random_search.best_params_
score = random_search.best_score_
print(params);print(score)

{'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 8, 8), 'activation': 'tanh'}
0.7746971270335756


In [1337]:
devrandom_search = RandomizedSearchCV(mlpdev, parameter_space,n_iter = 12, cv = 5)
devrandom_search.fit(X_develop,y_develop.ravel())
paramsdev = devrandom_search.best_params_
scoredev = devrandom_search.best_score_
print(paramsdev);print(scoredev)

{'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (5, 5, 5), 'activation': 'relu'}
0.6436559139784946


In [1338]:
print("test_score regression", accuracy_score(newy_test,regpred_test))

test_score regression 0.7445887445887446


In [1339]:
predict_test = random_search.predict(X_testnodev)
testscore = accuracy_score(y_testnodev,predict_test)
print("test_score normal model", testscore)

test_score normal model 0.7619047619047619


In [1340]:
predict_testdev = devrandom_search.predict(X_testnodev)
testscore = accuracy_score(y_test,predict_testdev)
print("test_score dev model", testscore)

test_score dev model 0.6147186147186147


In [1341]:
print("MATRIZ DE CONFUSAO DO TESTE DO MODELO SEM DESENVOLVIMENTO");print(confusion_matrix(y_test,predict_test))
print(classification_report(y_testnodev,predict_test))

MATRIZ DE CONFUSAO DO TESTE DO MODELO SEM DESENVOLVIMENTO
[[125  17]
 [ 38  51]]
              precision    recall  f1-score   support

           0       0.77      0.88      0.82       142
           1       0.75      0.57      0.65        89

    accuracy                           0.76       231
   macro avg       0.76      0.73      0.73       231
weighted avg       0.76      0.76      0.75       231



In [1355]:
print("MATRIZ DE CONFUSAO DO TESTE DO MODELO COM DESENVOLVIMENTO");print(confusion_matrix(y_test,predict_testdev))
print(classification_report(y_test,predict_testdev,zero_division=1))

MATRIZ DE CONFUSAO DO TESTE DO MODELO COM DESENVOLVIMENTO
[[142   0]
 [ 89   0]]
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       142
           1       1.00      0.00      0.00        89

    accuracy                           0.61       231
   macro avg       0.81      0.50      0.38       231
weighted avg       0.76      0.61      0.47       231

